In [ ]:
import babypandas as bpd
calfire = bpd.read_csv('data/calfire-full.csv')

# Applying Functions

Earlier, in Chapter 1, we saw that it is possible to create our own custom functions in Python. Such functions are very useful for repeatedly performing the same series of actions on different inputs. We have seen how to write functions that accept numbers and strings, but you'll be glad to know that they can accept *any* type of data, including DataFrames!

For example, suppose we frequently want to retrieve only those rows of a table whose entries lie between some thresholds. We
 might want only those fires in` calfire` from between 1995 and 2000, for instance. We can do so with a query:

In [ ]:
calfire[(calfire.get("year") >= 1995) & (calfire.get("year") < 2000)]

By writing this query into a function accepting a table, a column, and the thresholds, we make it easy to repeat. Such a function definition may look like:

In [ ]:
def between(table, column, start, stop):
    return table[(table.get(column) >= start) & (table.get(column) < stop)]

Then we can call our function to get only those fires from between 1995 and 2000:

In [ ]:
between(calfire, 'year', 1995, 2000)

Because this function accepts the column name, it is very reusable. We can use it to get the fires whose size is between 10,000 and 20,000 acres:

In [ ]:
between(calfire, 'acres', 10_000, 20_000)

Since the `<=` and `>` operators work on strings, too, we can get all of the fires whose name is between A and E:

In [ ]:
between(calfire, 'name', 'A', 'E')

## The `.apply` Series Method

DataFrames come equipped with many useful methods, but defining our own functions allows us to make tables even more powerful. One way to use tables with functions is to pass the table into the function as one of its inputs, as we saw in the example above. In some situations, however, we don't want to apply the function to the entire table, but rather to each entry in one of the table's columns. In these cases, we can use the `.apply` method.

For instance, suppose we have a table containing a `'year'` column, such as the `calfire` table we have been using, and we want to convert each year into the corresponding decade. We have already written a function that converts a single year to a decade: `decade_from_year`. Recall how it works:

In [ ]:
decade_from_year(1987)

We'd like to apply this function to each entry in the `'year'` column. To do so, we'll use `.apply`:

In [ ]:
calfire.get('year').apply(decade_from_year)

Notice the pattern here: we `.get('year')` to retrieve column we wish to work with, and then `.apply(decade_from_year)` to the column. The result is a Series with the same number of entries as the column containing the years. Each entry is the result of applying the function to the corresponding entry of the original column.

**Warning**

Note that we pass the function into `.apply` *without* trailing parentheses. That is, we write `.apply(decade_from_year)` and not `.apply(decade_from_year())` or `.apply(decade_from_year(calfire.get('year')))`. The `.apply` method accepts the name of a function. It will then call the function many times on the given Series.

In many cases we'd like to add this new Series back to the table as a new column. We can do so with `.assign`:

In [ ]:
with_decade = calfire.assign(
    decade=calfire.get('year').apply(decade_from_year)
)
with_decade

The `.apply` method is very useful for **data cleaning**. Data rarely comes to us in the exact form we need or prefer. For instance, we might wish to convert a year to its decade, or remove the leading number code from a fire's cause. A common approach to doing so is to write a function capable of converting or cleaning a single entry, then `.apply`ing this function to the entire column.

***Example: clean the `cause` column***

The `cause` column contains the cause of each fire as string, such as `'14 - Unknown'`. The string contains a number encoding unique to the cause of the fire, but this is redundant since the cause is described immediately after. Let's get rid of the number, leaving only the description.

First, we'll write a function that accepts a cause and returns only the description:

In [ ]:
def cause_description(cause):
    return cause.split('-')[-1].strip()

In [ ]:
cause_description('2 - Equipment Use')

Now we `.apply` the function to the `'cause'` column. We'll save it back to the table using `.assign`:

In [ ]:
calfire.assign(
    cause=calfire.get('cause').apply(cause_description)
)